# The Battle of Neigborhoods (Week 1)

## Introduction/Business Problem

__Madrid__ and __Barcelona__ are the largest cities in __Spain__. Madrid is the administrative capital and Barcelona the industrial one. The spanish __government__ is interested on __understanding how different or similar__ are these cities in order to study if they can __apply the same actions__ on both cities or not.

For a better understanding of the problem we __first__ study each city __individually__. __Then__, we __compere__ the results on both cities and will try to __find similarities__ between the cities or between districts of the different cities.

The annual __net income per capita__ is a good indicator to distinguish between the __rich and poor districts__ of the city. We can __also__ use as an indicator the __number of services that can generate a good quality of life__. A neighborhood may not have a high income but be well equipped. We are going to consider the following services as indicators of a good quality of life: hospitals and other medical centers, pharmacies, police stations, supermarkets, parks, playgrounds, Athletics & Sports venues, post offices, parking, event spaces, Arts & Entertainment venues, music venues, libraries, cultural centers, music schools and language schools(as an indicator of the education since there are schools and high schools in almost all districts).

## Data

### Data definition and source

In order to study this case we need first to __define the data__ that we need and __find__ the data __sources__. Then, we work on them to __create__ the __dataset__ that we will use for our purpose.

First, we are going to __study the income__ in each district for each city. So, we need to get the information about the income that are distributed geographically. Searching on internet, we found the __income for each postal code in Spain__ in 2018 being the data source the site https://www.epdata.es/ which is a portal (in spanish) that containts many studies and data sets on population, unemployment, income, crime and many other subjects. The data is stored on a __csv file__. To visualize this information on a map we need the __coordinates of each postal codes__. To do this we need to know the postal codes of each city, the information was found on the site http://es.postcode.info/ and then we get the coordinates using the geocoder python library for these postal codes.

Second, to study the venues in each district of the city. For this porpuse we will use the __FourSquare__ API to get the information needed. Knowing the coordinates of each district we can explore the selected venues of any type around.

In our study we will plot data about the different districts on a map, so we will need two __json files with the geographical districts limits__ for each city. We found some files on the internet but they need some modifications to fit our problem (this is why I'm not giving the source)

#### A) Incomes per district

In [1]:
import pandas as pd
import geocoder#get coordinates

After downloding the file with the annual mean income for all postal codes we load it on a pandas data frame to have a lok on the data.

In [2]:
spain_income_df = pd.read_csv('income_spain.csv')
spain_income_df.head()

,Year,Period,Parameter,Gross mean income,Net mean income
0,2018,Año,"02001-San Antonio Abad, La Pajarita, Polígono ...",32036,25823
1,2018,Año,"02002-Parque Sur, Huerta de Marzo (Albacete)",30426,24615
2,2018,Año,02003-Franciscanos (Albacete),26866,22145
3,2018,Año,"02004-Fátima, Villacerrada (Albacete)",23072,19449
4,2018,Año,"02005-Industria, El Pilar, Feria, Ntra Sra de ...",25098,20822


Since all the data is for the year 2018 we can drop the columns Period and Year because they don't give us any information. We also drop the Gross income column since we will be working on the __Net income__.

We can observe that the Parameter column has information about the __postal code__ and the __neighborhoods__ related to this postal code. We need to split this information in different columns to work with them later.

In [3]:
spain_income_df.drop(['Year', 'Period', 'Gross mean income'], axis=1, inplace=True)
spain_income_df[['postal_code','neighborhoods']] = spain_income_df.Parameter.str.split("-",expand=True)
spain_income_df.drop(['Parameter'], axis=1, inplace=True)

Now, __we need to related the postal codes with coordinates__ to be able later to plot on the maps. Here we difine a function that will help us to get the coordinates for each postal code.

In [4]:
def get_geocoder(postal_code):
    city = None
    if postal_code.startswith('280'):
        city = '{}, Madrid, Spain'
    else:
        city = '{}, Barcelona, Spain'
     # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis(city.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

We load and prepare the files with the postal codes for each city. Then, we merge separtly both of them with the income dataframe to create a __different dataframe for each city__.

In [5]:
postal_codes_madrid = pd.read_csv('Madrid_postal_codes_formatted.csv')
postal_codes_barcelona = pd.read_csv('Barcelona_postal_codes_formatted.csv')
postal_codes_madrid[['postal_code','dummy']] = postal_codes_madrid.postalcode.str.split("-",expand=True)
postal_codes_barcelona[['postal_code','dummy']] = postal_codes_barcelona.postalcode.str.split("-",expand=True)
postal_codes_madrid.drop(['dummy', 'postalcode'], axis=1, inplace=True)
postal_codes_barcelona.drop(['dummy', 'postalcode'], axis=1, inplace=True)

In [6]:
madrid_df = spain_income_df.merge(postal_codes_madrid, how='inner', left_on=['postal_code'], right_on=['postal_code'])
barcelona_df = spain_income_df.merge(postal_codes_barcelona, how='inner', left_on=['postal_code'], right_on=['postal_code'])

And we get the coordinates using the function previously defined

In [7]:
madrid_df['Latitude'], madrid_df['Longitude'] = zip(*madrid_df['postal_code'].apply(get_geocoder))
barcelona_df['Latitude'], barcelona_df['Longitude'] = zip(*barcelona_df['postal_code'].apply(get_geocoder))

In [8]:
madrid_df.head()

,Net mean income,postal_code,neighborhoods,city,Latitude,Longitude
0,71599,28001,"Salamanca, Goya (Madrid)",Madrid,40.424870,-3.682793
1,43426,28002,Chamartín (Madrid),Madrid,40.445035,-3.673343
2,41968,28003,"Chamberí, Rios Rosas (Madrid)",Madrid,40.442985,-3.705943
3,35353,28004,Justicia (Madrid),Madrid,40.424445,-3.699007
4,30078,28005,Arganzuela (Madrid),Madrid,40.405675,-3.712948


In [9]:
barcelona_df.head()

,Net mean income,postal_code,neighborhoods,city,Latitude,Longitude
0,19437,08001,"Ciutat Vella, El Raval (Barcelona)",Barcelona,41.380160,2.168716
1,26177,08002,Barrio Gótico (Barcelona),Barcelona,41.382180,2.176718
2,25768,08003,Barceloneta y Sant Pere (Barcelona),Barcelona,41.383210,2.187879
3,22526,08004,Pueblo Seco (Barcelona),Barcelona,41.370435,2.159972
4,30604,08005,El Poblenou (Barcelona),Barcelona,41.396235,2.201388


And we can store these data

In [10]:
#to store the data set on a csv file
madrid_df.to_csv("madrid_neig_coord.csv")
barcelona_df.to_csv("barcelona_neig_coord.csv")

We didn't find any site where were related the different postal codes to the districts. But we now which neigborhoods are in each districts. So we just include this information on the file where we have stored before. And __load a new file updated with the distircts__.

In [11]:
madird_w_districts = pd.read_csv('madrid_with_districts.csv')
barcelona_w_districts = pd.read_csv('barcelona_with_districts.csv')
madird_w_districts.drop(['Unnamed: 0', 'neighborhood'], axis =1, inplace=True)
barcelona_w_districts.drop(['Unnamed: 0', 'neighborhood'], axis =1, inplace=True)
madird_w_districts.sort_values(by='district', ascending=True, inplace=True)
barcelona_w_districts.sort_values(by='district', ascending=True, inplace=True)
madird_w_districts = madird_w_districts.groupby("district").mean()
barcelona_w_districts = barcelona_w_districts.groupby("district").mean()
madird_w_districts.reset_index(level=0,inplace=True)
barcelona_w_districts.reset_index(level=0,inplace=True)

In [12]:
madird_w_districts.head()

,district,Net mean income,Latitude,Longitude
0,Arganzuela,29817.000000,40.399677,-3.700679
1,Barajas,36363.000000,40.479245,-3.574264
2,Carabanchel,22859.333333,40.382017,-3.745679
3,Centro,32649.750000,40.420799,-3.704535
4,Chamartin,54030.000000,40.462981,-3.679981


In [13]:
barcelona_w_districts.head()

,district,Net mean income,Latitude,Longitude
0,Ciutat Vella,24702.000000,41.387212,2.181133
1,Eixample,38479.700000,41.392281,2.159918
2,Gracia,30870.333333,41.408921,2.152820
3,Horta Guinardo,17905.000000,41.425165,2.158162
4,Les Corts,48995.500000,41.386218,2.117649


#### B) Venues per district

For __each city__, we get __all the near defined venues__ in the introduction for __each districts__. Then we sump up the results and create a data frame with the total number of these venues in a district, and the district coordinates. 

The __url to hit the Foursquare API__ for each district will be like:

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            categoryid,
            LIMIT)

__For simplicity's sake We are not going to show the code, We are just going to load from a file where we have saved the information after doing all the data preparation.__

The __files__ with the total number of venues for each district:

In [14]:
Madrid_district_venues_df = pd.read_csv('madrid_data.csv')
Barcelona_district_venues_df = pd.read_csv('barcelona_data.csv')
Madrid_district_venues_df.drop(['Unnamed: 0'],axis=1,inplace=True)
Barcelona_district_venues_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [15]:
Madrid_district_venues_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,language school,park,athletic and sport,playground,parking,post office,supermarket,...,hospital,art and entertainment,library,medical center,police station,event space,cultural center,music venue,music school,total
0,Arganzuela,40.399677,-3.700679,4,9,25,6,6,4,17,...,0,39,6,28,5,7,3,6,1,175
1,Barajas,40.479245,-3.574264,0,0,2,1,1,1,0,...,0,0,0,7,0,2,1,0,1,19
2,Carabanchel,40.382017,-3.745679,0,6,4,0,0,3,3,...,8,4,1,10,4,3,0,7,0,61
3,Centro,40.420799,-3.704535,2,11,41,5,28,17,41,...,3,100,10,37,4,15,2,49,5,426
4,Chamartin,40.462981,-3.679981,3,5,13,2,3,6,6,...,5,5,0,33,2,7,3,2,0,100


In [16]:
Barcelona_district_venues_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,language school,park,athletic and sport,playground,parking,post office,supermarket,...,hospital,art and entertainment,library,medical center,police station,event space,cultural center,music venue,music school,total
0,Ciutat Vella,41.387212,2.181133,3,8,32,2,14,5,30,...,5,81,9,20,4,17,7,22,4,292
1,Eixample,41.392281,2.159918,10,14,43,6,26,8,37,...,15,57,5,40,9,20,5,17,3,372
2,Gracia,41.408921,2.152820,1,11,12,3,6,3,22,...,9,32,2,25,1,6,4,9,5,170
3,Horta Guinardo,41.425165,2.158162,4,6,6,1,0,3,5,...,4,4,1,6,0,3,4,3,2,69
4,Les Corts,41.386217,2.117649,4,12,20,3,8,3,8,...,8,3,1,27,0,3,2,1,1,113


And the files __to create the clusters__ with the top venues for each district. It is a similar process to the previous one but we extend the venues to other more general categories and we don't sum up the count of the different venue categories:

In [17]:
Madrid_grouped_df = pd.read_csv('madrid_grouped.csv')
Barcelona_grouped_df = pd.read_csv('barcelona_grouped.csv')
Madrid_grouped_df.drop(['Unnamed: 0'],axis=1,inplace=True)
Barcelona_grouped_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [18]:
Madrid_grouped_df.head()

,Neighbourhood,Alternative Healer,Amphitheater,Art Gallery,Art Museum,Arts & Entertainment,Athletics & Sports,Bar,Bookstore,Chiropractor,...,Skate Park,Skating Rink,Soccer Field,Spa,Sports Club,Supermarket,Tennis Court,Theater,Veterinarian,Yoga Studio
0,Arganzuela,0.000000,0.0,0.051429,0.011429,0.0,0.011429,0.00,0.040000,0.0,...,0.011429,0.005714,0.005714,0.0,0.000000,0.097143,0.005714,0.045714,0.017143,0.017143
1,Barajas,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.052632,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000
2,Carabanchel,0.000000,0.0,0.000000,0.000000,0.0,0.016393,0.00,0.032787,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,0.049180,0.000000,0.000000,0.016393,0.000000
3,Centro,0.004695,0.0,0.014085,0.018779,0.0,0.009390,0.00,0.093897,0.0,...,0.000000,0.000000,0.000000,0.0,0.002347,0.084507,0.000000,0.051643,0.004695,0.018779
4,Chamartin,0.000000,0.0,0.020000,0.000000,0.0,0.020000,0.01,0.010000,0.0,...,0.000000,0.010000,0.000000,0.0,0.000000,0.060000,0.000000,0.000000,0.020000,0.010000


In [19]:
Barcelona_grouped_df.head()

,Neighbourhood,Alternative Healer,Amphitheater,Art Gallery,Art Museum,Arts & Entertainment,Athletics & Sports,Bar,Bookstore,Chiropractor,...,Skate Park,Skating Rink,Soccer Field,Spa,Sports Club,Supermarket,Tennis Court,Theater,Veterinarian,Yoga Studio
0,Ciutat Vella,0.0,0.0,0.075342,0.017123,0.0,0.006849,0.010274,0.051370,0.003425,...,0.003425,0.006849,0.003425,0.000000,0.000000,0.099315,0.000000,0.027397,0.003425,0.017123
1,Eixample,0.0,0.0,0.037634,0.010753,0.0,0.008065,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.002688,0.002688,0.000000,0.091398,0.000000,0.018817,0.008065,0.018817
2,Gracia,0.0,0.0,0.023529,0.005882,0.0,0.011765,0.000000,0.052941,0.000000,...,0.000000,0.000000,0.017647,0.000000,0.000000,0.129412,0.005882,0.047059,0.005882,0.005882
3,Horta Guinardo,0.0,0.0,0.014493,0.000000,0.0,0.014493,0.000000,0.057971,0.000000,...,0.000000,0.000000,0.014493,0.000000,0.014493,0.072464,0.000000,0.000000,0.000000,0.000000
4,Les Corts,0.0,0.0,0.000000,0.000000,0.0,0.026549,0.000000,0.044248,0.000000,...,0.000000,0.000000,0.044248,0.000000,0.008850,0.061947,0.026549,0.000000,0.000000,0.000000
